In [1]:
import psycopg2
import pandas as pd
import numpy as np

from plotly.offline import init_notebook_mode, plot, iplot
from plotly.graph_objs import *
init_notebook_mode(connected=True)

In [2]:
hostname = '10.0.0.49'
username = 'scott'
password = 'diggler'
database = 'twitch'

In [3]:
def getQuery(query):
    c = psycopg2.connect( host=hostname, user=username, password=password, dbname=database )
    cur = c.cursor()
    cur.execute(query)
    return list(cur.fetchall())

# Find most popular game for each batch

In [4]:
%%time
query = """
SELECT s.batch AS batch,
       s.game AS game,
       v.max_views AS max_views
  FROM scrape AS s
JOIN
   (SELECT batch, 
           MAX(views) AS max_views
      FROM scrape
  GROUP BY batch
  ORDER BY batch) AS v
ON s.batch = v.batch AND
   s.views = v.max_views
"""

df = getQuery(query)
df = pd.DataFrame(df,columns=['date','game','views'])

CPU times: user 10.3 ms, sys: 3.87 ms, total: 14.1 ms
Wall time: 12.4 s


In [5]:
%%time
query="""
SELECT
    batch,
    first_game AS game,
    first_view AS views
FROM
    (SELECT
        batch,
        FIRST_VALUE(game) OVER (PARTITION BY batch ORDER BY views DESC) AS first_game,
        FIRST_VALUE(views) OVER (PARTITION BY batch ORDER BY views DESC) AS first_view
    FROM
        scrape
    GROUP BY
        batch,
        views,
        game) AS t
GROUP BY
    1,2,3
    ORDER BY batch DESC;
"""
df2 = getQuery(query)

df2 = pd.DataFrame(df2,
                  columns=['date','game','views'])

CPU times: user 7.6 ms, sys: 3.81 ms, total: 11.4 ms
Wall time: 55 s


In [85]:
df2 = df2.sort_values('date').reset_index(drop=True)

df = df.sort_values('date').reset_index(drop=True)

colors2 ="#5F9EA0 #4682B4 #B0C4DE #ADD8E6 #B0E0E6 #87CEFA #87CEEB #6495ED #00BFFF #1E90FF #4169E1 #0000FF #0000CD #00008B #000080 #191970 #FFF8DC #FFEBCD #FFE4C4 #FFDEAD #F5DEB3 #DEB887 #D2B48C #BC8F8F #F4A460 #DAA520 #B8860B #CD853F #D2691E #808000 #8B4513 #A0522D #A52A2A #800000".split()
gamedict = dict(zip(df.game.unique(),colors2[::2]))
gameshape = dict(zip(df.game.unique(),[101,102,106,107,108,116,118,119,320,121,321,135,143,110]))

In [86]:
iplot(
        {
    
            'data':[
                {'type':'scatter',
                'x':df['date'],
                'y':df['views'],
                'opacity':.7,
                 
                 'mode':'markers+lines',
                'marker':{'symbol':df['game'].map(lambda game: gameshape[game]),
                          'size':20,
                          'line':{'width':.2,
                                'color':'#17202A'},
                                  'color':df['game'].map(lambda game: gamedict[game])}}
            ]
    }
)

In [127]:
dg = df.groupby('game').sum()['views']
iplot(
        {
    
            'data':[
                {'type':'scatter',
                'x':dg.index,
                'y':dg,
                'opacity':.7,
                 'text':dg.index,
                 'mode':'markers+text',
                'marker':{'symbol':19,
                          'opacity':1,
                          'size':dg/1E6,
                          'line':{'width':.2,
                                'color':'#17202A'},
                                  'color':df['game'].map(lambda game: gamedict[game])}}
            ]
    }
)

In [95]:
dd = df.set_index('date')

In [109]:
DD = dd.to_period('D').groupby('game').max()

In [113]:
DD.head()

,views
game,
Counter-Strike: Global Offensive,254815
Dota 2,356483
E3,563303
E3 2018,1785348
Fortnite,1468529


In [130]:
gdict = dict(zip(DD.index,list(range(35))[::-1]))
             
iplot(
        {
    
            'data':[
                {'type':'scatter',
                'x':DD.index,
                'y':DD.views,
                'opacity':.7,
                 'text':DD.index,
                 'mode':'markers+text',
                'marker':{'symbol':DD.index.map(lambda game: gdict[game]),
                          'opacity':1,
                          'size':DD.views/1E4,
                          'line':{'width':.2,
                                'color':'#17202A'},
                                  'color':DD.index.map(lambda game: gamedict[game])}}
            ]
    }
)

In [ ]:
## day/day change for dota 2

In [137]:
query = """
SELECT game,
       EXTRACT(DAY FROM batch) AS day,
       EXTRACT(HOUR FROM batch) AS hour,
       views
FROM scrape
WHERE game = 'Dota 2'
LIMIT 100"""

df = pd.DataFrame(getQuery(query),columns=['game','day','hour','views'])

In [140]:
query = """
SELECT DATE(batch) AS day,
       COUNT(*) AS Frequency
       FROM scrape
       GROUP BY 1
       ORDER BY 1"""
pd.DataFrame(getQuery(query),columns=['day','Frequency'])

,day,Frequency
0,2018-06-03,9686
1,2018-06-04,34869
2,2018-06-05,68604
3,2018-06-06,56502
4,2018-06-07,114864
5,2018-06-08,115226
6,2018-06-09,110468
7,2018-06-10,113409
8,2018-06-11,119991
9,2018-06-12,117126


In [139]:
df.day.value_counts()

22.0    72
20.0    28
Name: day, dtype: int64

In [32]:
iplot({
    'data':[
        {'type':' bar',
        'x':df2['date'],
        'y':df2['views']}
    ]})

ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [{'y': 0       235619
1       217378
2       193738
3       254815
4       214667
5       228050
6       243841
7       213179
8       116860
9       180449
10      164979
11      125815
12      130624
13      128890
14      119008
15      172917
16      100582
17      138941
18      161501
19      153961
20      197753
21      221948
22      239624
23      281697
24      276651
25      284626
26      253889
27      221272
28      205416
29      188776
         ...  
1755    110994
1756    106734
1757    105305
1758    103019
1759    104345
1760    101940
1761     99516
1762     91512
1763     90516
1764     84329
1765     86704
1766     91945
1767     96189
1768     79559
1769     88126
1770     98160
1771    105015
1772    108288
1773    113734
1774     99276
1775     94866
1776    105163
1777    114818
1778    119809
1779    125318
1780    122777
1781    130833
1782    132991
1